# [Indexing and selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)

The axis labeling information in pandas objects serves many purposes:

* Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display.

* Enables automatic and explicit data alignment.

* Allows intuitive getting and setting of subsets of the data set.

In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects. The primary focus will be on Series and DataFrame as they have received more development attention in this area.

>Note
>
>The Python and NumPy indexing operators [] and attribute operator . provide quick and easy access to pandas data structures across a wide range of use cases. This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays. However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits. For production code, we recommended that you take advantage of the optimized pandas data access methods exposed in this chapter.

>Warning
>
>Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called chained assignment and should be avoided. See Returning a View versus Copy.

See the MultiIndex / Advanced Indexing for MultiIndex and more advanced indexing documentation.

See the cookbook for some advanced strategies.

## Different choices for indexing
Object selection has had a number of user-requested additions in order to support more explicit location based indexing. pandas now supports three types of multi-axis indexing.

* .loc is primarily label based, but may also be used with a boolean array. .loc will raise KeyError when the items are not found. Allowed inputs are:

  * A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).

  * A list or array of labels ['a', 'b', 'c'].

  * A slice object with labels 'a':'f' (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index! See Slicing with labels and Endpoints are inclusive.)

  * A boolean array (any NA values will be treated as False).

  * A callable function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).

See more at Selection by Label.

* .iloc is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a boolean array. .iloc will raise IndexError if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing. (this conforms with Python/NumPy slice semantics). Allowed inputs are:

  * A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).

  * A list or array of labels ['a', 'b', 'c'].

  * A slice object with labels 'a':'f' (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index! See Slicing with labels and Endpoints are inclusive.)

  * A boolean array (any NA values will be treated as False).

  * A callable function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).

See more at Selection by Label.

* .loc, .iloc, and also [] indexing can accept a callable as indexer. See more at Selection By Callable.

Getting values from an object with multi-axes selection uses the following notation (using .loc as an example, but the following applies to .iloc as well). Any of the axes accessors may be the null slice :. Axes left out of the specification are assumed to be :, e.g. p.loc['a'] is equivalent to p.loc['a', :, :].

| Object Type | Indexers |
|-------------|----------|
| Series | s.loc[indexer] |
| DataFrame | df.loc[row_indexer,column_indexer] |

## Basics
As mentioned when introducing the data structures in the last section, the primary function of indexing with [] (a.k.a. __getitem__ for those familiar with implementing class behavior in Python) is selecting out lower-dimensional slices. The following table shows return type values when indexing pandas objects with []:

| Object Type | Selection | Return Value Type |
|-------------|-----------|-------------------|
| Series | series[label] | scalar value |
| DataFrame | frame[colname] | Series corresponding to colname |

Here we construct a simple time series data set to use for illustrating the indexing functionality:

In [1]:
import shutil
from distutils.dir_util import copy_tree
copy_tree('archive/','data/')

['data/baseball.xlsb',
 'data/mindex_ex.csv',
 'data/banklist.html',
 'data/baseball.csv',
 'data/iris.data',
 'data/spam.html']

In [2]:
import pandas as pd
import numpy as np
import sys

In [3]:
dates = pd.date_range('1/1/2000', periods=8)
df1 = pd.DataFrame(np.random.randn(8, 4),
                  index=dates, columns=['A', 'B', 'C', 'D'])
df1

,A,B,C,D
2000-01-01,-0.074987,-1.256942,0.242585,-0.448856
2000-01-02,-0.026471,0.677206,0.439110,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-04,0.719355,-0.240973,-0.246007,0.160804
2000-01-05,-0.540646,-0.541343,0.181085,-0.007486
2000-01-06,0.515638,2.383790,-0.838101,0.789838
2000-01-07,2.504488,-0.211760,0.308634,-1.091179
2000-01-08,-0.243090,-0.972391,0.745060,-0.590304


>Note
>
>None of the indexing functionality is time series specific unless specifically stated.

Thus, as per above, we have the most basic indexing using []:

In [4]:
s1 = df1['A']
s1[dates[5]]

0.5156382534407756

You can pass a list of columns to [] to select columns in that order. If a column is not contained in the DataFrame, an exception will be raised. Multiple columns can also be set in this manner:

In [5]:
df2 = df1.copy()
df2

,A,B,C,D
2000-01-01,-0.074987,-1.256942,0.242585,-0.448856
2000-01-02,-0.026471,0.677206,0.439110,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-04,0.719355,-0.240973,-0.246007,0.160804
2000-01-05,-0.540646,-0.541343,0.181085,-0.007486
2000-01-06,0.515638,2.383790,-0.838101,0.789838
2000-01-07,2.504488,-0.211760,0.308634,-1.091179
2000-01-08,-0.243090,-0.972391,0.745060,-0.590304


In [6]:
df2[['B', 'A']] = df1[['A', 'B']]
df2

,A,B,C,D
2000-01-01,-1.256942,-0.074987,0.242585,-0.448856
2000-01-02,0.677206,-0.026471,0.439110,-1.646895
2000-01-03,-0.062915,-1.128684,-0.369834,-0.673542
2000-01-04,-0.240973,0.719355,-0.246007,0.160804
2000-01-05,-0.541343,-0.540646,0.181085,-0.007486
2000-01-06,2.383790,0.515638,-0.838101,0.789838
2000-01-07,-0.211760,2.504488,0.308634,-1.091179
2000-01-08,-0.972391,-0.243090,0.745060,-0.590304


You may find this useful for applying a transform (in-place) to a subset of the columns.

>Warning
>
>pandas aligns all AXES when setting Series and DataFrame from .loc, and .iloc.
>
>This will not modify df because the column alignment is before value assignment.

In [7]:
df2[['A', 'B']]

,A,B
2000-01-01,-1.256942,-0.074987
2000-01-02,0.677206,-0.026471
2000-01-03,-0.062915,-1.128684
2000-01-04,-0.240973,0.719355
2000-01-05,-0.541343,-0.540646
2000-01-06,2.383790,0.515638
2000-01-07,-0.211760,2.504488
2000-01-08,-0.972391,-0.243090


In [8]:
df2.loc[:, ['B', 'A']] = df2[['A', 'B']]
df2[['A', 'B']]

,A,B
2000-01-01,-1.256942,-0.074987
2000-01-02,0.677206,-0.026471
2000-01-03,-0.062915,-1.128684
2000-01-04,-0.240973,0.719355
2000-01-05,-0.541343,-0.540646
2000-01-06,2.383790,0.515638
2000-01-07,-0.211760,2.504488
2000-01-08,-0.972391,-0.243090


The correct way to swap column values is by using raw values:

In [9]:
df2.loc[:, ['B', 'A']] = df2[['A', 'B']].to_numpy()
df2[['A', 'B']]

,A,B
2000-01-01,-0.074987,-1.256942
2000-01-02,-0.026471,0.677206
2000-01-03,-1.128684,-0.062915
2000-01-04,0.719355,-0.240973
2000-01-05,-0.540646,-0.541343
2000-01-06,0.515638,2.383790
2000-01-07,2.504488,-0.211760
2000-01-08,-0.243090,-0.972391


## Attribute access
You may access an index on a Series or column on a DataFrame directly as an attribute:

In [10]:
s3 = pd.Series([1, 2, 3], index=list('abc'))
df3 = df1.copy()
s3.b

2

In [11]:
df3.A

2000-01-01   -0.074987
2000-01-02   -0.026471
2000-01-03   -1.128684
2000-01-04    0.719355
2000-01-05   -0.540646
2000-01-06    0.515638
2000-01-07    2.504488
2000-01-08   -0.243090
Freq: D, Name: A, dtype: float64

In [12]:
s3.a = 5
s3

a    5
b    2
c    3
dtype: int64

In [13]:
df3.A = list(range(len(df3.index)))  # ok if A already exists
df3

,A,B,C,D
2000-01-01,0,-1.256942,0.242585,-0.448856
2000-01-02,1,0.677206,0.439110,-1.646895
2000-01-03,2,-0.062915,-0.369834,-0.673542
2000-01-04,3,-0.240973,-0.246007,0.160804
2000-01-05,4,-0.541343,0.181085,-0.007486
2000-01-06,5,2.383790,-0.838101,0.789838
2000-01-07,6,-0.211760,0.308634,-1.091179
2000-01-08,7,-0.972391,0.745060,-0.590304


In [14]:
df3['AA'] = list(range(len(df3.index)))  # use this form to create a new column
df3

,A,B,C,D,AA
2000-01-01,0,-1.256942,0.242585,-0.448856,0
2000-01-02,1,0.677206,0.439110,-1.646895,1
2000-01-03,2,-0.062915,-0.369834,-0.673542,2
2000-01-04,3,-0.240973,-0.246007,0.160804,3
2000-01-05,4,-0.541343,0.181085,-0.007486,4
2000-01-06,5,2.383790,-0.838101,0.789838,5
2000-01-07,6,-0.211760,0.308634,-1.091179,6
2000-01-08,7,-0.972391,0.745060,-0.590304,7


>Warning
>
>* You can use this access only if the index element is a valid Python identifier, e.g. s.1 is not allowed. See here for an explanation of valid identifiers.
>
>* The attribute will not be available if it conflicts with an existing method name, e.g. s.min is not allowed, but s['min'] is possible.
>
>* Similarly, the attribute will not be available if it conflicts with any of the following list: index, major_axis, minor_axis, items.
>
>* In any of these cases, standard indexing will still work, e.g. s['1'], s['min'], and s['index'] will access the corresponding element or column.

If you are using the IPython environment, you may also use tab-completion to see these accessible attributes.

You can also assign a dict to a row of a DataFrame:

In [15]:
x4 = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})
x4.iloc[1] = {'x': 9, 'y': 99}
x4

,x,y
0,1,3
1,9,99
2,3,5


You can use attribute access to modify an existing element of a Series or column of a DataFrame, but be careful; if you try to use attribute access to create a new column, it creates a new attribute rather than a new column. In 0.21.0 and later, this will raise a UserWarning:

In [16]:
df4 = pd.DataFrame({'one': [1., 2., 3.]})
df4.two = [4, 5, 6]
df4

<ipython-input-16-8503afd9ff10>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df4.two = [4, 5, 6]


,one
0,1.0
1,2.0
2,3.0


## Slicing ranges
The most robust and consistent way of slicing ranges along arbitrary axes is described in the Selection by Position section detailing the .iloc method. For now, we explain the semantics of slicing using the [] operator.

With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels:

In [17]:
s4 = df3.B.copy()
s4[:5]

2000-01-01   -1.256942
2000-01-02    0.677206
2000-01-03   -0.062915
2000-01-04   -0.240973
2000-01-05   -0.541343
Freq: D, Name: B, dtype: float64

In [18]:
s4[::2]

2000-01-01   -1.256942
2000-01-03   -0.062915
2000-01-05   -0.541343
2000-01-07   -0.211760
Freq: 2D, Name: B, dtype: float64

In [19]:
s4[::-1]

2000-01-08   -0.972391
2000-01-07   -0.211760
2000-01-06    2.383790
2000-01-05   -0.541343
2000-01-04   -0.240973
2000-01-03   -0.062915
2000-01-02    0.677206
2000-01-01   -1.256942
Freq: -1D, Name: B, dtype: float64

Note that setting works as well:

In [20]:
s5 = s4.copy()
s5[:5] = 0
s5

2000-01-01    0.000000
2000-01-02    0.000000
2000-01-03    0.000000
2000-01-04    0.000000
2000-01-05    0.000000
2000-01-06    2.383790
2000-01-07   -0.211760
2000-01-08   -0.972391
Freq: D, Name: B, dtype: float64

With DataFrame, slicing inside of [] slices the rows. This is provided largely as a convenience since it is such a common operation.

In [21]:
df1[:3]

,A,B,C,D
2000-01-01,-0.074987,-1.256942,0.242585,-0.448856
2000-01-02,-0.026471,0.677206,0.439110,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542


In [22]:
df1[::-1]

,A,B,C,D
2000-01-08,-0.243090,-0.972391,0.745060,-0.590304
2000-01-07,2.504488,-0.211760,0.308634,-1.091179
2000-01-06,0.515638,2.383790,-0.838101,0.789838
2000-01-05,-0.540646,-0.541343,0.181085,-0.007486
2000-01-04,0.719355,-0.240973,-0.246007,0.160804
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-02,-0.026471,0.677206,0.439110,-1.646895
2000-01-01,-0.074987,-1.256942,0.242585,-0.448856


## Selection by label
>Warning
>
>Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called chained assignment and should be avoided. See Returning a View versus Copy.

>Warning
>
>.loc is strict when you present slicers that are not compatible (or convertible) with the index type. For example using integers in a DatetimeIndex. These will raise a TypeError.

In [23]:
df6 = pd.DataFrame(np.random.randn(5, 4),
                   columns=list('ABCD'),
                   index=pd.date_range('20130101', periods=5))
df6

,A,B,C,D
2013-01-01,0.788290,0.632719,-0.413555,1.112892
2013-01-02,1.118844,0.631641,-0.927500,0.350331
2013-01-03,-0.208465,-1.560978,1.148182,1.200744
2013-01-04,0.446009,-2.679845,0.328961,0.170593
2013-01-05,0.720404,0.922716,1.374592,-0.264449


In [24]:
try:
    df6.loc[2:3]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'TypeError: cannot do slice indexing on DatetimeIndex with these indexers [2] of type int \n'
    msg = msg + 'FIX: use a compatible index\n'
    print(msg)    

sys.exc_info(): <class 'TypeError'> 
TypeError: cannot do slice indexing on DatetimeIndex with these indexers [2] of type int 
FIX: use a compatible index



String likes in slicing can be convertible to the type of the index and lead to natural slicing.

In [25]:
df6.loc['20130102':'20130104']

,A,B,C,D
2013-01-02,1.118844,0.631641,-0.927500,0.350331
2013-01-03,-0.208465,-1.560978,1.148182,1.200744
2013-01-04,0.446009,-2.679845,0.328961,0.170593


>Warning
>
>Changed in version 1.0.0.
>
>pandas will raise a KeyError if indexing with a list with missing labels. See list-like Using loc with missing keys in a list is Deprecated.

pandas provides a suite of methods in order to have purely label based indexing. This is a strict inclusion based protocol. Every label asked for must be in the index, or a KeyError will be raised. When slicing, both the start bound AND the stop bound are included, if present in the index. Integers are valid labels, but they refer to the label and not the position.

The .loc attribute is the primary access method. The following are valid inputs:

* A single label, e.g. 5 or 'a' (Note that 5 is interpreted as a label of the index. This use is not an integer position along the index.).

* A list or array of labels ['a', 'b', 'c'].

* A slice object with labels 'a':'f' (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index! See Slicing with labels.

* A boolean array.

* A callable, see Selection By Callable.

In [26]:
s6 = pd.Series(np.random.randn(6), index=list('abcdef'))
s6

a    1.075027
b    0.313985
c   -0.158443
d    1.243521
e   -0.285159
f   -0.982653
dtype: float64

In [27]:
s6.loc['c':]

c   -0.158443
d    1.243521
e   -0.285159
f   -0.982653
dtype: float64

In [28]:
s6.loc['b']

0.3139854426517362

Note that setting works as well:

In [29]:
s6.loc['c':] = 0
s6

a    1.075027
b    0.313985
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

With a DataFrame:

In [30]:
df7 = pd.DataFrame(np.random.randn(6, 4),
                   index=list('abcdef'),
                   columns=list('ABCD'))
df7

,A,B,C,D
a,0.473391,-0.749853,-0.255315,-2.163724
b,1.231482,-0.218517,0.530492,1.136916
c,-0.541129,-0.053888,-0.828884,-0.632101
d,0.146490,-1.316671,1.032930,0.043182
e,0.287797,-0.795837,1.057077,-0.409538
f,-1.001562,-1.432905,1.023503,1.476649


In [31]:
df7.loc[['a', 'b', 'd'], :]

,A,B,C,D
a,0.473391,-0.749853,-0.255315,-2.163724
b,1.231482,-0.218517,0.530492,1.136916
d,0.146490,-1.316671,1.032930,0.043182


Accessing via label slices:

In [32]:
df7.loc['d':, 'A':'C']

,A,B,C
d,0.146490,-1.316671,1.032930
e,0.287797,-0.795837,1.057077
f,-1.001562,-1.432905,1.023503


For getting a cross section using a label (equivalent to df.xs('a')):

In [33]:
df7.loc['a']

A    0.473391
B   -0.749853
C   -0.255315
D   -2.163724
Name: a, dtype: float64

For getting values with a boolean array:

In [34]:
df7.loc['a'] > 1

A    False
B    False
C    False
D    False
Name: a, dtype: bool

In [35]:
df7.loc[:, df7.loc['a'] > 1]

""
a
b
c
d
e
f


NA values in a boolean array propagate as False:

*Changed in version 1.0.2.*

In [36]:
mask7 = pd.array([True, False, True, False, pd.NA, False], dtype="boolean")
mask7

<BooleanArray>
[True, False, True, False, <NA>, False]
Length: 6, dtype: boolean

In [37]:
df7[mask7]

,A,B,C,D
a,0.473391,-0.749853,-0.255315,-2.163724
c,-0.541129,-0.053888,-0.828884,-0.632101


For getting a value explicitly:

In [38]:
# this is also equivalent to ``df7.at['a','A']``
df7.loc['a', 'A']

0.4733909371943915

### Slicing with labels
When using .loc with slices, if both the start and the stop labels are present in the index, then elements located between the two (including them) are returned:

In [39]:
s8 = pd.Series(list('abcde'), index=[0, 3, 2, 5, 4])
s8.loc[3:5]

3    b
2    c
5    d
dtype: object

If at least one of the two is absent, but the index is sorted, and can be compared against start and stop labels, then slicing will still work as expected, by selecting labels which rank between the two:

In [40]:
s8.sort_index()

0    a
2    c
3    b
4    e
5    d
dtype: object

In [41]:
s8.sort_index().loc[1:6]

2    c
3    b
4    e
5    d
dtype: object

However, if at least one of the two is absent and the index is not sorted, an error will be raised (since doing otherwise would be computationally expensive, as well as potentially ambiguous for mixed type indexes). For instance, in the above example, s.loc[1:6] would raise KeyError.

For the rationale behind this behavior, see Endpoints are inclusive.

In [42]:
s9 = pd.Series(list('abcdef'), index=[0, 3, 2, 5, 4, 2])
s9.loc[3:5]

3    b
2    c
5    d
dtype: object

Also, if the index has duplicate labels and either the start or the stop label is dupulicated, an error will be raised. For instance, in the above example, s.loc[2:5] would raise a KeyError.

For more information about duplicate labels, see Duplicate Labels.

## Selection by position
>Warning
>
>Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called chained assignment and should be avoided. See Returning a View versus Copy.

pandas provides a suite of methods in order to get purely integer based indexing. The semantics follow closely Python and NumPy slicing. These are 0-based indexing. When slicing, the start bound is included, while the upper bound is excluded. Trying to use a non-integer, even a valid label will raise an IndexError.

The .iloc attribute is the primary access method. The following are valid inputs:

* An integer e.g. 5.

* A list or array of integers [4, 3, 0].

* A slice object with ints 1:7.

* A boolean array.

* A callable, see Selection By Callable.

In [43]:
s10 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))
s10

0   -0.484767
2   -1.421807
4    0.673546
6    0.331638
8    0.270397
dtype: float64

In [44]:
s10.iloc[:3]

0   -0.484767
2   -1.421807
4    0.673546
dtype: float64

In [45]:
s10.iloc[3]

0.3316378221560643

Note that setting works as well:

In [46]:
s10.iloc[:3] = 0
s10

0    0.000000
2    0.000000
4    0.000000
6    0.331638
8    0.270397
dtype: float64

With a DataFrame:

In [47]:
df11 = pd.DataFrame(np.random.randn(6, 4),
                   index=list(range(0, 12, 2)),
                   columns=list(range(0, 8, 2)))
df11

,0,2,4,6
0,0.332356,-0.293997,0.799851,0.952210
2,-1.995351,-1.764400,0.234691,-1.334128
4,0.317948,-0.222478,-0.062057,-1.942821
6,0.456425,-0.242453,-0.108255,1.053144
8,0.346076,0.151017,-0.872187,1.277678
10,0.771636,0.848568,0.046110,-1.645438


Select via integer slicing:

In [48]:
df11.iloc[:3]

,0,2,4,6
0,0.332356,-0.293997,0.799851,0.952210
2,-1.995351,-1.764400,0.234691,-1.334128
4,0.317948,-0.222478,-0.062057,-1.942821


In [49]:
df11.iloc[1:5, 2:4]

,4,6
2,0.234691,-1.334128
4,-0.062057,-1.942821
6,-0.108255,1.053144
8,-0.872187,1.277678


Select via integer list:

In [50]:
df11.iloc[[1, 3, 5], [1, 3]]

,2,6
2,-1.764400,-1.334128
6,-0.242453,1.053144
10,0.848568,-1.645438


In [51]:
df11.iloc[1:3, :]

,0,2,4,6
2,-1.995351,-1.764400,0.234691,-1.334128
4,0.317948,-0.222478,-0.062057,-1.942821


In [52]:
df11.iloc[:, 1:3]

,2,4
0,-0.293997,0.799851
2,-1.764400,0.234691
4,-0.222478,-0.062057
6,-0.242453,-0.108255
8,0.151017,-0.872187
10,0.848568,0.046110


In [53]:
# this is also equivalent to ``df1.iat[1,1]``
df11.iloc[1, 1]

-1.7643997240452869

For getting a cross section using an integer position (equiv to df.xs(1)):

In [54]:
df11.iloc[1]

0   -1.995351
2   -1.764400
4    0.234691
6   -1.334128
Name: 2, dtype: float64

Out of range slice indexes are handled gracefully just as in Python/NumPy.

In [55]:
# these are allowed in Python/NumPy.
x11 = list('abcdef')
x11

['a', 'b', 'c', 'd', 'e', 'f']

In [56]:
x11[4:10]

['e', 'f']

In [57]:
x11[8:10]

[]

In [58]:
s11 = pd.Series(x11)
s11

0    a
1    b
2    c
3    d
4    e
5    f
dtype: object

In [59]:
s11.iloc[4:10]

4    e
5    f
dtype: object

In [60]:
s11.iloc[8:10]

Series([], dtype: object)

Note that using slices that go out of bounds can result in an empty axis (e.g. an empty DataFrame being returned).

In [61]:
df12 = pd.DataFrame(np.random.randn(5, 2), columns=list('AB'))
df12

,A,B
0,-0.294322,-0.043660
1,-0.907588,1.394037
2,0.174637,-0.761605
3,-3.137060,-0.827254
4,0.310462,-0.219157


In [62]:
df12.iloc[:, 2:3]

""
0
1
2
3
4


In [63]:
df12.iloc[:, 2:3]

""
0
1
2
3
4


In [64]:
df12.iloc[:, 1:3]

,B
0,-0.043660
1,1.394037
2,-0.761605
3,-0.827254
4,-0.219157


In [65]:
df12.iloc[4:6]

,A,B
4,0.310462,-0.219157


A single indexer that is out of bounds will raise an IndexError. A list of indexers where any element is out of bounds will raise an IndexError.

In [66]:
try:
    df12.iloc[[4, 5, 6]]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'IndexError: positional indexers are out-of-bounds \n'
    msg = msg + 'FIX: use correct index range \n'
    print(msg)

sys.exc_info(): <class 'IndexError'> 
IndexError: positional indexers are out-of-bounds 
FIX: use correct index range 



In [67]:
try:
    df12.iloc[:, 4]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'IndexError: single positional indexer is out-of-bounds \n'
    msg = msg + 'FIX: use correct index range \n'
    print(msg)

sys.exc_info(): <class 'IndexError'> 
IndexError: single positional indexer is out-of-bounds 
FIX: use correct index range 



## Selection by callable
.loc, .iloc, and also [] indexing can accept a callable as indexer. The callable must be a function with one argument (the calling Series or DataFrame) that returns valid output for indexing.

In [68]:
df13 = pd.DataFrame(np.random.randn(6, 4),
                    index=list('abcdef'),
                    columns=list('ABCD'))
df13

,A,B,C,D
a,-1.849923,-0.736301,0.124714,-0.410841
b,-1.583109,-0.184772,-1.057701,0.425149
c,0.201784,-0.404077,-1.745443,1.431990
d,1.438440,0.057268,0.420775,0.976834
e,-1.051490,0.702170,0.436866,-0.323575
f,0.365970,1.309234,-0.668312,-0.894901


In [69]:
df13.loc[lambda dfx: dfx['A'] > 0, :]

,A,B,C,D
c,0.201784,-0.404077,-1.745443,1.431990
d,1.438440,0.057268,0.420775,0.976834
f,0.365970,1.309234,-0.668312,-0.894901


In [70]:
df13.loc[:, lambda dfx: ['A', 'B']]

,A,B
a,-1.849923,-0.736301
b,-1.583109,-0.184772
c,0.201784,-0.404077
d,1.438440,0.057268
e,-1.051490,0.702170
f,0.365970,1.309234


In [71]:
df13.iloc[:, lambda dfx: [0, 1]]

,A,B
a,-1.849923,-0.736301
b,-1.583109,-0.184772
c,0.201784,-0.404077
d,1.438440,0.057268
e,-1.051490,0.702170
f,0.365970,1.309234


In [72]:
df13[lambda dfx: dfx.columns[0]]

a   -1.849923
b   -1.583109
c    0.201784
d    1.438440
e   -1.051490
f    0.365970
Name: A, dtype: float64

You can use callable indexing in Series.

In [73]:
df13['A'].loc[lambda sx: sx > 0]

c    0.201784
d    1.438440
f    0.365970
Name: A, dtype: float64

Using these methods / indexers, you can chain data selection operations without using a temporary variable.

In [74]:
bb13 = pd.read_csv('data/baseball.csv', index_col='id')
(bb13.groupby(['year', 'team']).sum().loc[lambda dfx: dfx['r'] > 100])

stint    g    ab    r    h  X2b  X3b  hr    rbi    sb   cs   bb  \
year team                                                                    
2007 CIN       6  379   745  101  203   35    2  36  125.0  10.0  1.0  105   
     DET       5  301  1062  162  283   54    4  37  144.0  24.0  7.0   97   
     HOU       4  311   926  109  218   47    6  14   77.0  10.0  4.0   60   
     LAN      11  413  1021  153  293   61    3  36  154.0   7.0  5.0  114   
     NYN      13  622  1854  240  509  101    3  61  243.0  22.0  4.0  174   
     SFN       5  482  1305  198  337   67    6  40  171.0  26.0  7.0  235   
     TEX       2  198   729  115  200   40    4  28  115.0  21.0  4.0   73   
     TOR       4  459  1408  187  378   96    2  58  223.0   4.0  2.0  190   

              so   ibb   hbp    sh    sf  gidp  
year team                                       
2007 CIN   127.0  14.0   1.0   1.0  15.0  18.0  
     DET   176.0   3.0  10.0   4.0   8.0  28.0  
     HOU   212.0   3.0   9.0  16.0   6.0  17.0  
     LAN   141.0   8.0   9.0   3.0   8.0  29.0  
     NYN   310.0  24.0  23.0  18.0  15.0  48.0  
     SFN   188.0  51.0   8.0  16.0   6.0  41.0  
     TEX   140.0   4.0   5.0   2.0   8.0  16.0  
     TOR   265.0  16.0  12.0   4.0  16.0  38.0

## Combining positional and label-based indexing
If you wish to get the 0th and the 2nd elements from the index in the ‘A’ column, you can do:

In [75]:
df14 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6]},
                   index=list('abc'))
df14

,A,B
a,1,4
b,2,5
c,3,6


In [76]:
df14.loc[df14.index[[0, 2]], 'A']

a    1
c    3
Name: A, dtype: int64

This can also be expressed using .iloc, by explicitly getting locations on the indexers, and using positional indexing to select things.

In [77]:
df14.iloc[[0, 2], df14.columns.get_loc('A')]

a    1
c    3
Name: A, dtype: int64

For getting multiple indexers, using .get_indexer:

In [78]:
df14.iloc[[0, 2], df14.columns.get_indexer(['A', 'B'])]

,A,B
a,1,4
c,3,6


## Indexing with list with missing labels is deprecated
>Warning
>
>Changed in version 1.0.0.
>
>Using .loc or [] with a list with one or more missing labels will no longer reindex, in favor of .reindex.

In prior versions, using .loc[list-of-labels] would work as long as at least 1 of the keys was found (otherwise it would raise a KeyError). This behavior was changed and will now raise a KeyError if at least one label is missing. The recommended alternative is to use .reindex().

For example.

In [79]:
s14 = pd.Series([1, 2, 3])
s14

0    1
1    2
2    3
dtype: int64

Selection with all keys found is unchanged.

In [80]:
s14.loc[[1, 2]]

1    2
2    3
dtype: int64

Previous behavior

In [81]:
try:
    s14.loc[[1, 2, 3]]
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'Passing list-likes to .loc with any non-matching elements will raise \n'
    msg = msg + 'KeyError in the future, you can use .reindex() as an alternative. \n'
    msg = msg + 'FIX: use correct index range \n'
    print(msg)

sys.exc_info(): <class 'KeyError'> 
Passing list-likes to .loc with any non-matching elements will raise 
KeyError in the future, you can use .reindex() as an alternative. 
FIX: use correct index range 



See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike

### Reindexing
The idiomatic way to achieve selecting potentially not-found elements is via .reindex(). See also the section on reindexing.

In [82]:
s14.reindex([1, 2, 3])

1    2.0
2    3.0
3    NaN
dtype: float64

Alternatively, if you want to select only valid keys, the following is idiomatic and efficient; it is guaranteed to preserve the dtype of the selection.

In [83]:
labels = [1, 2, 3]
s14.loc[s14.index.intersection(labels)]

1    2
2    3
dtype: int64

Having a duplicated index will raise for a .reindex():

In [84]:
s14 = pd.Series(np.arange(4), index=['a', 'a', 'b', 'c'])
labels = ['c', 'd']

In [85]:
try:
    s14.reindex(labels)
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'ValueError: cannot reindex from a duplicate axis \n'
    msg = msg + 'FIX: only reindex primary index \n'
    print(msg)

sys.exc_info(): <class 'ValueError'> 
ValueError: cannot reindex from a duplicate axis 
FIX: only reindex primary index 



Generally, you can intersect the desired labels with the current axis, and then reindex.

In [86]:
s14.loc[s14.index.intersection(labels)].reindex(labels)

c    3.0
d    NaN
dtype: float64

However, this would still raise if your resulting index is duplicated.

In [87]:
try:
    s14.loc[s14.index.intersection(labels)].reindex(labels)
except:
    msg = 'sys.exc_info(): {} \n'.format(sys.exc_info()[0] )
    msg = msg + 'ValueError: cannot reindex from a duplicate axis \n'
    msg = msg + 'FIX: only reindex primary index \n'
    print(msg)

## Selecting random samples
A random selection of rows or columns from a Series or DataFrame with the sample() method. The method will sample rows by default, and accepts a specific number of rows/columns to return, or a fraction of rows.

In [88]:
s15 = pd.Series([0, 1, 2, 3, 4, 5])

# When no arguments are passed, returns 1 row.
s15.sample()

5    5
dtype: int64

In [89]:
# One may specify either a number of rows:
s15.sample(n=3)

0    0
2    2
4    4
dtype: int64

In [90]:
# Or a fraction of the rows:
s15.sample(frac=0.5)

4    4
3    3
5    5
dtype: int64

By default, sample will return each row at most once, but one can also sample with replacement using the replace option:

In [91]:
s16 = pd.Series([0, 1, 2, 3, 4, 5])
# Without replacement (default):
s16.sample(n=6, replace=False)

1    1
0    0
2    2
4    4
5    5
3    3
dtype: int64

In [92]:
# With replacement:
s16.sample(n=6, replace=True)

5    5
1    1
3    3
2    2
4    4
2    2
dtype: int64

By default, each row has an equal probability of being selected, but if you want rows to have different probabilities, you can pass the sample function sampling weights as weights. These weights can be a list, a NumPy array, or a Series, but they must be of the same length as the object you are sampling. Missing values will be treated as a weight of zero, and inf values are not allowed. If weights do not sum to 1, they will be re-normalized by dividing all weights by the sum of the weights. For example:

In [93]:
s17 = pd.Series([0, 1, 2, 3, 4, 5])
example_weights = [0, 0, 0.2, 0.2, 0.2, 0.4]
s17.sample(n=3, weights=example_weights)

5    5
3    3
4    4
dtype: int64

In [94]:
# Weights will be re-normalized automatically
example_weights2 = [0.5, 0, 0, 0, 0, 0]
s17.sample(n=1, weights=example_weights2)

0    0
dtype: int64

When applied to a DataFrame, you can use a column of the DataFrame as sampling weights (provided you are sampling rows and not columns) by simply passing the name of the column as a string.

In [95]:
df17 = pd.DataFrame({'col1': [9, 8, 7, 6],
                     'weight_column': [0.5, 0.4, 0.1, 0]})
df17.sample(n=3, weights='weight_column')

,col1,weight_column
1,8,0.4
0,9,0.5
2,7,0.1


sample also allows users to sample columns instead of rows using the axis argument.

In [96]:
df18 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [2, 3, 4]})
df18.sample(n=1, axis=1)

,col2
0,2
1,3
2,4


Finally, one can also set a seed for sample’s random number generator using the random_state argument, which will accept either an integer (as a seed) or a NumPy RandomState object.

In [97]:
df19 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [2, 3, 4]})

# With a given seed, the sample will always draw the same rows.
df19.sample(n=2, random_state=2)

,col1,col2
2,3,4
1,2,3


In [98]:
df19.sample(n=2, random_state=2)

,col1,col2
2,3,4
1,2,3


## Setting with enlargement
The .loc/[] operations can perform enlargement when setting a non-existent key for that axis.

In the Series case this is effectively an appending operation.

In [99]:
s20 = pd.Series([1, 2, 3])
s20

0    1
1    2
2    3
dtype: int64

In [100]:
s20[5] = 5.
s20

0    1.0
1    2.0
2    3.0
5    5.0
dtype: float64

A DataFrame can be enlarged on either axis via .loc.

In [101]:
df20 = pd.DataFrame(np.arange(6).reshape(3, 2),
                   columns=['A', 'B'])
df20

,A,B
0,0,1
1,2,3
2,4,5


In [102]:
df20.loc[:, 'C'] = df20.loc[:, 'A']
df20

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4


This is like an append operation on the DataFrame.

In [103]:
df20.loc[3] = 5
df20

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4
3,5,5,5


## Fast scalar value getting and setting
Since indexing with [] must handle a lot of cases (single-label access, slicing, boolean indexing, etc.), it has a bit of overhead in order to figure out what you’re asking for. If you only want to access a scalar value, the fastest way is to use the at and iat methods, which are implemented on all of the data structures.

Similarly to loc, at provides label based scalar lookups, while, iat provides integer based lookups analogously to iloc

In [104]:
s17.iat[5]

5

In [105]:
df21 = df1.copy()
df21.at[dates[5], 'A']

0.5156382534407756

In [106]:
df21.iat[3, 0]

0.7193548344796457

You can also set using these same indexers.

In [107]:
df21.at[dates[5], 'E'] = 7
df21.iat[3, 0] = 7

at may enlarge the object in-place as above if the indexer is missing.

In [108]:
df21.at[dates[-1] + pd.Timedelta('1 day'), 0] = 7
df21

,A,B,C,D,E,0
2000-01-01,-0.074987,-1.256942,0.242585,-0.448856,NaN,NaN
2000-01-02,-0.026471,0.677206,0.439110,-1.646895,NaN,NaN
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542,NaN,NaN
2000-01-04,7.000000,-0.240973,-0.246007,0.160804,NaN,NaN
2000-01-05,-0.540646,-0.541343,0.181085,-0.007486,NaN,NaN
2000-01-06,0.515638,2.383790,-0.838101,0.789838,7.0,NaN
2000-01-07,2.504488,-0.211760,0.308634,-1.091179,NaN,NaN
2000-01-08,-0.243090,-0.972391,0.745060,-0.590304,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN,NaN,7.0


## Boolean indexing
Another common operation is the use of boolean vectors to filter the data. The operators are: | for or, & for and, and ~ for not. These must be grouped by using parentheses, since by default Python will evaluate an expression such as df['A'] > 2 & df['B'] < 3 as df['A'] > (2 & df['B']) < 3, while the desired evaluation order is (df['A'] > 2) & (df['B'] < 3).

Using a boolean vector to index a Series works exactly as in a NumPy ndarray:

In [109]:
s22 = pd.Series(range(-3, 4))
s22

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [110]:
s22[s22 > 0]

4    1
5    2
6    3
dtype: int64

In [111]:
s22[(s22 < -1) | (s22 > 0.5)]

0   -3
1   -2
4    1
5    2
6    3
dtype: int64

In [112]:
s22[~(s22 < 0)]

3    0
4    1
5    2
6    3
dtype: int64

You may select rows from a DataFrame using a boolean vector the same length as the DataFrame’s index (for example, something derived from one of the columns of the DataFrame):

In [113]:
df22 = df21.copy()
df22[df22['A'] > 0]

,A,B,C,D,E,0
2000-01-04,7.000000,-0.240973,-0.246007,0.160804,NaN,NaN
2000-01-06,0.515638,2.383790,-0.838101,0.789838,7.0,NaN
2000-01-07,2.504488,-0.211760,0.308634,-1.091179,NaN,NaN


List comprehensions and the map method of Series can also be used to produce more complex criteria:

In [114]:
df23 = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                     'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                     'c': np.random.randn(7)})

# only want 'two' or 'three'
criterion = df23['a'].map(lambda x: x.startswith('t'))
df23[criterion]

,a,b,c
2,two,y,0.581766
3,three,x,1.213923
4,two,y,-0.106434


In [115]:
# equivalent but slower
df23[[x.startswith('t') for x in df23['a']]]

,a,b,c
2,two,y,0.581766
3,three,x,1.213923
4,two,y,-0.106434


In [116]:
# Multiple criteria
df23[criterion & (df23['b'] == 'x')]

,a,b,c
3,three,x,1.213923


With the choice methods Selection by Label, Selection by Position, and Advanced Indexing you may select along more than one axis using boolean vectors combined with other indexing expressions.

In [117]:
df23.loc[criterion & (df23['b'] == 'x'), 'b':'c']

,b,c
3,x,1.213923


>Warning
>
>iloc supports two kinds of boolean indexing. If the indexer is a boolean Series, an error will be raised. For instance, in the following example, df.iloc[s.values, 1] is ok. The boolean indexer is an array. But df.iloc[s, 1] would raise ValueError.

In [118]:
df24 = pd.DataFrame([[1, 2], [3, 4], [5, 6]],
                    index=list('abc'),
                    columns=['A', 'B'])
s24 = (df24['A'] > 2)
s24

a    False
b     True
c     True
Name: A, dtype: bool

In [119]:
df24.loc[s24, 'B']

b    4
c    6
Name: B, dtype: int64

In [120]:
df24.iloc[s24.values, 1]

b    4
c    6
Name: B, dtype: int64

## Indexing with isin
Consider the isin() method of Series, which returns a boolean vector that is true wherever the Series elements exist in the passed list. This allows you to select rows where one or more columns have values you want:

In [121]:
s25 = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
s25

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [122]:
s25.isin([2, 4, 6])

4    False
3    False
2     True
1    False
0     True
dtype: bool

In [123]:
s25[s25.isin([2, 4, 6])]

2    2
0    4
dtype: int64

The same method is available for Index objects and is useful for the cases when you don’t know which of the sought labels are in fact present:

In [124]:
s25[s25.index.isin([2, 4, 6])]

4    0
2    2
dtype: int64

In [125]:
# compare it to the following
s25.reindex([2, 4, 6])

2    2.0
4    0.0
6    NaN
dtype: float64

In addition to that, MultiIndex allows selecting a separate level to use in the membership check:

In [126]:
s26 = pd.Series(np.arange(6),
                 index=pd.MultiIndex.from_product([[0, 1], ['a', 'b', 'c']]))
s26

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int64

In [127]:
s26.iloc[s26.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])]

0  c    2
1  a    3
dtype: int64

In [128]:
s26.iloc[s26.index.isin(['a', 'c', 'e'], level=1)]

0  a    0
   c    2
1  a    3
   c    5
dtype: int64

DataFrame also has an isin() method. When calling isin, pass a set of values as either an array or dict. If values is an array, isin returns a DataFrame of booleans that is the same shape as the original DataFrame, with True wherever the element is in the sequence of values.

In [129]:
df27 = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                     'ids2': ['a', 'n', 'c', 'n']})
values = ['a', 'b', 1, 3]
df27.isin(values)

,vals,ids,ids2
0,True,True,True
1,False,True,False
2,True,False,False
3,False,False,False


Oftentimes you’ll want to match certain values with certain columns. Just make values a dict where the key is the column, and the value is a list of items you want to check for.

In [130]:
values = {'ids': ['a', 'b'], 'vals': [1, 3]}
df27.isin(values)

,vals,ids,ids2
0,True,True,False
1,False,True,False
2,True,False,False
3,False,False,False


Combine DataFrame’s isin with the any() and all() methods to quickly select subsets of your data that meet a given criteria. To select a row where each column meets its own criterion:

In [131]:
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}
row_mask = df27.isin(values).all(1)
df27[row_mask]

,vals,ids,ids2
0,1,a,a


## The where() Method and Masking
Selecting values from a Series with a boolean vector generally returns a subset of the data. To guarantee that selection output has the same shape as the original data, you can use the where method in Series and DataFrame.

To return only the selected rows:

In [132]:
s25[s25 > 0]

3    1
2    2
1    3
0    4
dtype: int64

To return a Series of the same shape as the original:

In [133]:
s25.where(s25 > 0)

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

Selecting values from a DataFrame with a boolean criterion now also preserves input data shape. where is used under the hood as the implementation. The code below is equivalent to df.where(df < 0).

In [134]:
df1[df1 < 0]

,A,B,C,D
2000-01-01,-0.074987,-1.256942,NaN,-0.448856
2000-01-02,-0.026471,NaN,NaN,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-04,NaN,-0.240973,-0.246007,NaN
2000-01-05,-0.540646,-0.541343,NaN,-0.007486
2000-01-06,NaN,NaN,-0.838101,NaN
2000-01-07,NaN,-0.211760,NaN,-1.091179
2000-01-08,-0.243090,-0.972391,NaN,-0.590304


In addition, where takes an optional other argument for replacement of values where the condition is False, in the returned copy.

In [135]:
df1.where(df1 < 0, -df1)

,A,B,C,D
2000-01-01,-0.074987,-1.256942,-0.242585,-0.448856
2000-01-02,-0.026471,-0.677206,-0.439110,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-04,-0.719355,-0.240973,-0.246007,-0.160804
2000-01-05,-0.540646,-0.541343,-0.181085,-0.007486
2000-01-06,-0.515638,-2.383790,-0.838101,-0.789838
2000-01-07,-2.504488,-0.211760,-0.308634,-1.091179
2000-01-08,-0.243090,-0.972391,-0.745060,-0.590304


You may wish to set values based on some boolean criteria. This can be done intuitively like so:

In [136]:
s26 = s25.copy()
s26[s26 < 0] = 0
s26

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [137]:
df26 = df1.copy()
df26[df26 < 0] = 0
df26

,A,B,C,D
2000-01-01,0.000000,0.000000,0.242585,0.000000
2000-01-02,0.000000,0.677206,0.439110,0.000000
2000-01-03,0.000000,0.000000,0.000000,0.000000
2000-01-04,0.719355,0.000000,0.000000,0.160804
2000-01-05,0.000000,0.000000,0.181085,0.000000
2000-01-06,0.515638,2.383790,0.000000,0.789838
2000-01-07,2.504488,0.000000,0.308634,0.000000
2000-01-08,0.000000,0.000000,0.745060,0.000000


By default, where returns a modified copy of the data. There is an optional parameter inplace so that the original data can be modified without creating a copy:

In [138]:
df27 = df1.copy()
df27.where(df1 > 0, -df1, inplace=True)
df27

,A,B,C,D
2000-01-01,0.074987,1.256942,0.242585,0.448856
2000-01-02,0.026471,0.677206,0.439110,1.646895
2000-01-03,1.128684,0.062915,0.369834,0.673542
2000-01-04,0.719355,0.240973,0.246007,0.160804
2000-01-05,0.540646,0.541343,0.181085,0.007486
2000-01-06,0.515638,2.383790,0.838101,0.789838
2000-01-07,2.504488,0.211760,0.308634,1.091179
2000-01-08,0.243090,0.972391,0.745060,0.590304


>Note
>
>The signature for DataFrame.where() differs from numpy.where(). Roughly df1.where(m, df2) is equivalent to np.where(m, df1, df2).

In [139]:
df1.where(df1 < 0, -df1) == np.where(df1 < 0, df1, -df1)

,A,B,C,D
2000-01-01,True,True,True,True
2000-01-02,True,True,True,True
2000-01-03,True,True,True,True
2000-01-04,True,True,True,True
2000-01-05,True,True,True,True
2000-01-06,True,True,True,True
2000-01-07,True,True,True,True
2000-01-08,True,True,True,True


#### Alignment

Furthermore, where aligns the input boolean condition (ndarray or DataFrame), such that partial selection with setting is possible. This is analogous to partial setting via .loc (but on the contents rather than the axis labels).

In [140]:
df28 = df1.copy()
df28[df28[1:4] > 0] = 3
df28

,A,B,C,D
2000-01-01,-0.074987,-1.256942,0.242585,-0.448856
2000-01-02,-0.026471,3.000000,3.000000,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-04,3.000000,-0.240973,-0.246007,3.000000
2000-01-05,-0.540646,-0.541343,0.181085,-0.007486
2000-01-06,0.515638,2.383790,-0.838101,0.789838
2000-01-07,2.504488,-0.211760,0.308634,-1.091179
2000-01-08,-0.243090,-0.972391,0.745060,-0.590304


Where can also accept axis and level parameters to align the input when performing the where.

In [141]:
df28 = df1.copy()
df29 = df1.copy()
df28

,A,B,C,D
2000-01-01,-0.074987,-1.256942,0.242585,-0.448856
2000-01-02,-0.026471,0.677206,0.439110,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-04,0.719355,-0.240973,-0.246007,0.160804
2000-01-05,-0.540646,-0.541343,0.181085,-0.007486
2000-01-06,0.515638,2.383790,-0.838101,0.789838
2000-01-07,2.504488,-0.211760,0.308634,-1.091179
2000-01-08,-0.243090,-0.972391,0.745060,-0.590304


In [142]:
df28.where(df28 > 0, df28['A'], axis='index')

,A,B,C,D
2000-01-01,-0.074987,-0.074987,0.242585,-0.074987
2000-01-02,-0.026471,0.677206,0.439110,-0.026471
2000-01-03,-1.128684,-1.128684,-1.128684,-1.128684
2000-01-04,0.719355,0.719355,0.719355,0.160804
2000-01-05,-0.540646,-0.540646,0.181085,-0.540646
2000-01-06,0.515638,2.383790,0.515638,0.789838
2000-01-07,2.504488,2.504488,0.308634,2.504488
2000-01-08,-0.243090,-0.243090,0.745060,-0.243090


This is equivalent to (but faster than) the following.

In [143]:
df29.apply(lambda x, y: x.where(x > 0, y), y=df29['A'])

,A,B,C,D
2000-01-01,-0.074987,-0.074987,0.242585,-0.074987
2000-01-02,-0.026471,0.677206,0.439110,-0.026471
2000-01-03,-1.128684,-1.128684,-1.128684,-1.128684
2000-01-04,0.719355,0.719355,0.719355,0.160804
2000-01-05,-0.540646,-0.540646,0.181085,-0.540646
2000-01-06,0.515638,2.383790,0.515638,0.789838
2000-01-07,2.504488,2.504488,0.308634,2.504488
2000-01-08,-0.243090,-0.243090,0.745060,-0.243090


where can accept a callable as condition and other arguments. The function must be with one argument (the calling Series or DataFrame) and that returns valid output as condition and other argument.

In [144]:
df30 = pd.DataFrame({'A': [1, 2, 3],
                     'B': [4, 5, 6],
                     'C': [7, 8, 9]})
df30.where(lambda x: x > 4, lambda x: x + 10)

,A,B,C
0,11,14,7
1,12,5,8
2,13,6,9


### Mask
mask() is the inverse boolean operation of where.

In [145]:
s25.mask(s25 >= 0)

4   NaN
3   NaN
2   NaN
1   NaN
0   NaN
dtype: float64

In [146]:
df1.mask(df1 >= 0)

,A,B,C,D
2000-01-01,-0.074987,-1.256942,NaN,-0.448856
2000-01-02,-0.026471,NaN,NaN,-1.646895
2000-01-03,-1.128684,-0.062915,-0.369834,-0.673542
2000-01-04,NaN,-0.240973,-0.246007,NaN
2000-01-05,-0.540646,-0.541343,NaN,-0.007486
2000-01-06,NaN,NaN,-0.838101,NaN
2000-01-07,NaN,-0.211760,NaN,-1.091179
2000-01-08,-0.243090,-0.972391,NaN,-0.590304


## Setting with enlargement conditionally using numpy()
An alternative to where() is to use numpy.where(). Combined with setting a new column, you can use it to enlarge a dataframe where the values are determined conditionally.

Consider you have two choices to choose from in the following dataframe. And you want to set a new column color to ‘green’ when the second column has ‘Z’. You can do the following:

In [147]:
df32 = pd.DataFrame({'col1': list('ABBC'), 'col2': list('ZZXY')})
df32['color'] = np.where(df32['col2'] == 'Z', 'green', 'red')
df32

,col1,col2,color
0,A,Z,green
1,B,Z,green
2,B,X,red
3,C,Y,red


If you have multiple conditions, you can use numpy.select() to achieve that. Say corresponding to three conditions there are three choice of colors, with a fourth color as a fallback, you can do the following.

In [148]:
conditions = [
    (df32['col2'] == 'Z') & (df32['col1'] == 'A'),
    (df32['col2'] == 'Z') & (df32['col1'] == 'B'),
    (df32['col1'] == 'B')
]

choices = ['yellow', 'blue', 'purple']
df32['color'] = np.select(conditions, choices, default='black')
df32

,col1,col2,color
0,A,Z,yellow
1,B,Z,blue
2,B,X,purple
3,C,Y,black


## The query() Method
DataFrame objects have a query() method that allows selection using an expression.

You can get the value of the frame where column b has values between the values of columns a and c. For example:

In [149]:
n = 10
df33 = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df33

,a,b,c
0,0.593987,0.940530,0.068396
1,0.595395,0.870116,0.169629
2,0.213100,0.852541,0.335604
3,0.098898,0.494780,0.302166
4,0.751052,0.580389,0.754131
5,0.173614,0.796932,0.066503
6,0.006921,0.881154,0.215218
7,0.982791,0.380319,0.402644
8,0.359626,0.234309,0.271891
9,0.277642,0.094344,0.024861


In [150]:
# pure python
df33[(df33['a'] < df33['b']) & (df33['b'] < df33['c'])]

,a,b,c


In [151]:
# query
df33.query('(a < b) & (b < c)')

,a,b,c


Do the same thing but fall back on a named index if there is no column with the name a.

In [152]:
df34 = pd.DataFrame(np.random.randint(n / 2, size=(n, 2)), columns=list('bc'))
df34.index.name = 'a'
df34

,b,c
a,,
0,2,0
1,0,0
2,0,3
3,1,0
4,0,1
5,2,1
6,1,4
7,4,4
8,3,2


In [153]:
df34.query('a < b and b < c')

,b,c
a,,


If instead you don’t want to or cannot name your index, you can use the name index in your query expression:

In [154]:
df35 = pd.DataFrame(np.random.randint(n, size=(n, 2)), columns=list('bc'))
df35

,b,c
0,0,7
1,4,7
2,1,3
3,4,9
4,6,2
5,7,6
6,1,1
7,1,9
8,8,1
9,2,6


In [155]:
df35.query('index < b < c')

,b,c
1,4,7
3,4,9


>Note
>
>If the name of your index overlaps with a column name, the column name is given precedence. For example,

In [156]:
df36 = pd.DataFrame({'a': np.random.randint(5, size=5)})
df36.index.name = 'a'
df36.query('a > 2')  # uses the column 'a', not the index

,a
a,
4,4


>You can still use the index in a query expression by using the special identifier ‘index’:

In [157]:
df36.query('index > 2')

,a
a,
3,0
4,4


>If for some reason you have a column named index, then you can refer to the index as ilevel_0 as well, but at this point you should consider renaming your columns to something less ambiguous.

## MultiIndex query() Syntax
You can also use the levels of a DataFrame with a MultiIndex as if they were columns in the frame:

In [158]:
n = 10
colors = np.random.choice(['red', 'green'], size=n)
foods = np.random.choice(['eggs', 'ham'], size=n)
colors

array(['green', 'red', 'red', 'red', 'red', 'red', 'red', 'red', 'red',
       'green'], dtype='<U5')

In [159]:
foods

array(['ham', 'ham', 'eggs', 'eggs', 'eggs', 'ham', 'eggs', 'ham', 'ham',
       'ham'], dtype='<U4')

In [160]:
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])
df = pd.DataFrame(np.random.randn(n, 2), index=index)
df

0         1
color food                    
green ham  -0.373553 -0.110182
red   ham  -0.427507 -0.589417
      eggs -0.130320  0.443852
      eggs  0.689254 -0.668267
      eggs  1.301308  1.906223
      ham  -0.408636 -0.263882
      eggs  0.238067  1.179072
      ham   0.405713  1.832662
      ham   0.556699 -0.077662
green ham   1.126763  0.255281

In [161]:
df.query('color == "red"')

0         1
color food                    
red   ham  -0.427507 -0.589417
      eggs -0.130320  0.443852
      eggs  0.689254 -0.668267
      eggs  1.301308  1.906223
      ham  -0.408636 -0.263882
      eggs  0.238067  1.179072
      ham   0.405713  1.832662
      ham   0.556699 -0.077662

If the levels of the MultiIndex are unnamed, you can refer to them using special names:

In [162]:
df.index.names = [None, None]
df

0         1
green ham  -0.373553 -0.110182
red   ham  -0.427507 -0.589417
      eggs -0.130320  0.443852
      eggs  0.689254 -0.668267
      eggs  1.301308  1.906223
      ham  -0.408636 -0.263882
      eggs  0.238067  1.179072
      ham   0.405713  1.832662
      ham   0.556699 -0.077662
green ham   1.126763  0.255281

In [163]:
df.query('ilevel_0 == "red"')

0         1
red ham  -0.427507 -0.589417
    eggs -0.130320  0.443852
    eggs  0.689254 -0.668267
    eggs  1.301308  1.906223
    ham  -0.408636 -0.263882
    eggs  0.238067  1.179072
    ham   0.405713  1.832662
    ham   0.556699 -0.077662

The convention is ilevel_0, which means “index level 0” for the 0th level of the index.

## query() Use Cases
A use case for query() is when you have a collection of DataFrame objects that have a subset of column names (or index levels/names) in common. You can pass the same query to both frames without having to specify which frame you’re interested in querying

In [164]:
df37 = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df37

,a,b,c
0,0.221859,0.615015,0.597141
1,0.696010,0.947325,0.230843
2,0.656996,0.772783,0.924126
3,0.056133,0.444834,0.101346
4,0.881805,0.535938,0.076703
5,0.810628,0.217745,0.703927
6,0.626359,0.509920,0.527122
7,0.704534,0.594689,0.906607
8,0.957897,0.370941,0.117860
9,0.406975,0.691782,0.542387


In [166]:
df38 = pd.DataFrame(np.random.rand(n + 2, 3), columns=df37.columns)
df38

,a,b,c
0,0.747914,0.801034,0.013214
1,0.773522,0.485262,0.615882
2,0.961025,0.036006,0.584339
3,0.969800,0.542599,0.654458
4,0.463633,0.735110,0.561906
5,0.867691,0.094530,0.874625
6,0.868128,0.237729,0.441578
7,0.874840,0.299919,0.085157
8,0.685165,0.472796,0.022921
9,0.341007,0.478671,0.458327


In [167]:
expr = '0.0 <= a <= c <= 0.5'
map(lambda frame: frame.query(expr), [df36, df37])

## query() Python versus pandas Syntax Comparison
Full numpy-like syntax:

In [168]:
df39 = pd.DataFrame(np.random.randint(n, size=(n, 3)), columns=list('abc'))
df39

,a,b,c
0,6,1,3
1,1,6,8
2,7,1,6
3,3,6,5
4,0,3,5
5,4,0,0
6,2,4,6
7,9,8,2
8,6,7,2
9,6,9,4


In [169]:
df39.query('(a < b) & (b < c)')

,a,b,c
1,1,6,8
4,0,3,5
6,2,4,6


In [170]:
df39[(df39['a'] < df39['b']) & (df39['b'] < df39['c'])]

,a,b,c
1,1,6,8
4,0,3,5
6,2,4,6


Slightly nicer by removing the parentheses (by binding making comparison operators bind tighter than & and |).

In [171]:
df39.query('a < b & b < c')

,a,b,c
1,1,6,8
4,0,3,5
6,2,4,6


Use English instead of symbols:

In [172]:
df39.query('a < b and b < c')

,a,b,c
1,1,6,8
4,0,3,5
6,2,4,6


Pretty close to how you might write it on paper:

In [173]:
df39.query('a < b < c')

,a,b,c
1,1,6,8
4,0,3,5
6,2,4,6


## The `in` and `not in` operators
query() also supports special use of Python’s in and not in comparison operators, providing a succinct syntax for calling the isin method of a Series or DataFrame.

In [174]:
# get all rows where columns "a" and "b" have overlapping values
df40 = pd.DataFrame({'a': list('aabbccddeeff'), 'b': list('aaaabbbbcccc'),
                     'c': np.random.randint(5, size=12),
                     'd': np.random.randint(9, size=12)})
df40

,a,b,c,d
0,a,a,4,1
1,a,a,0,0
2,b,a,0,1
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0
6,d,b,0,7
7,d,b,0,4
8,e,c,1,3
9,e,c,4,0


In [175]:
df40.query('a in b')

,a,b,c,d
0,a,a,4,1
1,a,a,0,0
2,b,a,0,1
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0


In [178]:
# How you'd do it in pure Python
df40[df40['a'].isin(df40['b'])]

,a,b,c,d
0,a,a,4,1
1,a,a,0,0
2,b,a,0,1
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0


In [179]:
df40.query('a not in b')

,a,b,c,d
6,d,b,0,7
7,d,b,0,4
8,e,c,1,3
9,e,c,4,0
10,f,c,4,4
11,f,c,2,7


In [182]:
# pure Python
df40[~df40['a'].isin(df40['b'])]

,a,b,c,d
6,d,b,0,7
7,d,b,0,4
8,e,c,1,3
9,e,c,4,0
10,f,c,4,4
11,f,c,2,7


You can combine this with other expressions for very succinct queries:

In [183]:
# rows where cols a and b have overlapping values
# and col c's values are less than col d's
df40.query('a in b and c < d')

,a,b,c,d
2,b,a,0,1
3,b,a,2,8


In [184]:
# pure Python
df40[df40['b'].isin(df40['a']) & (df40['c'] < df40['d'])]

,a,b,c,d
2,b,a,0,1
3,b,a,2,8
6,d,b,0,7
7,d,b,0,4
8,e,c,1,3
11,f,c,2,7


>Note
>
>Note that in and not in are evaluated in Python, since numexpr has no equivalent of this operation. However, only the in/not in expression itself is evaluated in vanilla Python. For example, in the expression
```python
df.query('a in b + c + d')
```
>(b + c + d) is evaluated by numexpr and then the in operation is evaluated in plain Python. In general, any operations that can be evaluated using numexpr will be.

## Special use of the == operator with list objects
Comparing a list of values to a column using ==/!= works similarly to in/not in.

In [186]:
df40.query('b == ["a", "b", "c"]')

,a,b,c,d
0,a,a,4,1
1,a,a,0,0
2,b,a,0,1
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0
6,d,b,0,7
7,d,b,0,4
8,e,c,1,3
9,e,c,4,0


In [187]:
# pure Python
df40[df40['b'].isin(["a", "b", "c"])]

,a,b,c,d
0,a,a,4,1
1,a,a,0,0
2,b,a,0,1
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0
6,d,b,0,7
7,d,b,0,4
8,e,c,1,3
9,e,c,4,0


In [188]:
df40.query('c == [1, 2]')

,a,b,c,d
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0
8,e,c,1,3
11,f,c,2,7


In [189]:
df40.query('c != [1, 2]')

,a,b,c,d
0,a,a,4,1
1,a,a,0,0
2,b,a,0,1
6,d,b,0,7
7,d,b,0,4
9,e,c,4,0
10,f,c,4,4


In [190]:
# using in/not in
df40.query('[1, 2] in c')

,a,b,c,d
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0
8,e,c,1,3
11,f,c,2,7


In [191]:
df40.query('[1, 2] not in c')

,a,b,c,d
0,a,a,4,1
1,a,a,0,0
2,b,a,0,1
6,d,b,0,7
7,d,b,0,4
9,e,c,4,0
10,f,c,4,4


In [192]:
# pure Python
df40[df40['c'].isin([1, 2])]

,a,b,c,d
3,b,a,2,8
4,c,b,1,0
5,c,b,1,0
8,e,c,1,3
11,f,c,2,7


## Boolean operators
You can negate boolean expressions with the word not or the ~ operator.

In [193]:
df41 = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df41['bools'] = np.random.rand(len(df41)) > 0.5
df41.query('~bools')

,a,b,c,bools
0,0.282361,0.379913,0.852456,False
2,0.552867,0.457879,0.673178,False
4,0.002717,0.116012,0.340023,False
5,0.686067,0.778435,0.559402,False
7,0.902772,0.169258,0.959648,False
8,0.766151,0.322043,0.443383,False
9,0.366408,0.545162,0.805863,False


In [194]:
df41.query('not bools')

,a,b,c,bools
0,0.282361,0.379913,0.852456,False
2,0.552867,0.457879,0.673178,False
4,0.002717,0.116012,0.340023,False
5,0.686067,0.778435,0.559402,False
7,0.902772,0.169258,0.959648,False
8,0.766151,0.322043,0.443383,False
9,0.366408,0.545162,0.805863,False


In [195]:
df41.query('not bools') == df41[~df41['bools']]

,a,b,c,bools
0,True,True,True,True
2,True,True,True,True
4,True,True,True,True
5,True,True,True,True
7,True,True,True,True
8,True,True,True,True
9,True,True,True,True


Of course, expressions can be arbitrarily complex too:

In [198]:
# short query syntax
In [276]: shorter = df41.query('a < b < c and (not bools) or bools > 2')

# equivalent in pure Python
longer = df41[(df41['a'] < df41['b'])
            & (df41['b'] < df41['c'])
            & (~df41['bools'])
            | (df41['bools'] > 2)]
shorter

,a,b,c,bools
0,0.282361,0.379913,0.852456,False
4,0.002717,0.116012,0.340023,False
9,0.366408,0.545162,0.805863,False


In [199]:
longer

,a,b,c,bools
0,0.282361,0.379913,0.852456,False
4,0.002717,0.116012,0.340023,False
9,0.366408,0.545162,0.805863,False


In [200]:
shorter == longer

,a,b,c,bools
0,True,True,True,True
4,True,True,True,True
9,True,True,True,True


Performance of query()
DataFrame.query() using numexpr is slightly faster than Python for large frames.

![query-perf.png](img/query-perf.png)
>Note
>
>You will only see the performance benefits of using the numexpr engine with DataFrame.query() if your frame has more than approximately 200,000 rows.
>
>![query-perf-small.png](img/query-perf-small.png)

This plot was created using a DataFrame with 3 columns each containing floating point values generated using numpy.random.randn().

## Duplicate data
If you want to identify and remove duplicate rows in a DataFrame, there are two methods that will help: duplicated and drop_duplicates. Each takes as an argument the columns to use to identify duplicated rows.

* duplicated returns a boolean vector whose length is the number of rows, and which indicates whether a row is duplicated.

* drop_duplicates removes duplicate rows.

By default, the first observed row of a duplicate set is considered unique, but each method has a keep parameter to specify targets to be kept.

* keep='first' (default): mark / drop duplicates except for the first occurrence.

* keep='last': mark / drop duplicates except for the last occurrence.

* keep=False: mark / drop all duplicates.

In [201]:
df42 = pd.DataFrame({'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
                     'b': ['x', 'y', 'x', 'y', 'x', 'x', 'x'],
                     'c': np.random.randn(7)})
df42

,a,b,c
0,one,x,-0.313102
1,one,y,3.030697
2,two,x,-1.066584
3,two,y,0.887840
4,two,x,1.538934
5,three,x,0.746125
6,four,x,-0.620357


In [202]:
df42.duplicated('a')

0    False
1     True
2    False
3     True
4     True
5    False
6    False
dtype: bool

In [203]:
df42.duplicated('a', keep='last')

0     True
1    False
2     True
3     True
4    False
5    False
6    False
dtype: bool

In [204]:
df42.duplicated('a', keep=False)

0     True
1     True
2     True
3     True
4     True
5    False
6    False
dtype: bool

In [205]:
df42.drop_duplicates('a')

,a,b,c
0,one,x,-0.313102
2,two,x,-1.066584
5,three,x,0.746125
6,four,x,-0.620357


In [206]:
df42.drop_duplicates('a', keep='last')

,a,b,c
1,one,y,3.030697
4,two,x,1.538934
5,three,x,0.746125
6,four,x,-0.620357


In [207]:
df42.drop_duplicates('a', keep=False)

,a,b,c
5,three,x,0.746125
6,four,x,-0.620357


Also, you can pass a list of columns to identify duplications.

In [208]:
df42.duplicated(['a', 'b'])

0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [209]:
df42.drop_duplicates(['a', 'b'])

,a,b,c
0,one,x,-0.313102
1,one,y,3.030697
2,two,x,-1.066584
3,two,y,0.887840
5,three,x,0.746125
6,four,x,-0.620357


To drop duplicates by index value, use Index.duplicated then perform slicing. The same set of options are available for the keep parameter.

In [210]:
df43 = pd.DataFrame({'a': np.arange(6),
                     'b': np.random.randn(6)},
                    index=['a', 'a', 'b', 'c', 'b', 'a'])
df43

,a,b
a,0,0.578073
a,1,2.350474
b,2,0.534074
c,3,-0.232980
b,4,-1.027731
a,5,-0.133433


In [211]:
df43.index.duplicated()

array([False,  True, False, False,  True,  True])

In [213]:
df43[~df43.index.duplicated()]

,a,b
a,0,0.578073
b,2,0.534074
c,3,-0.232980


In [214]:
df43[~df43.index.duplicated(keep='last')]

,a,b
c,3,-0.232980
b,4,-1.027731
a,5,-0.133433


In [215]:
df43[~df43.index.duplicated(keep=False)]

,a,b
c,3,-0.23298


## Dictionary-like get() method
Each of Series or DataFrame have a get method which can return a default value.

In [216]:
s44 = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
s44.get('a')  # equivalent to s['a']

1

In [217]:
s44.get('x', default=-1)

-1

## Looking up values by index/column labels
Sometimes you want to extract a set of values given a sequence of row labels and column labels, this can be achieved by DataFrame.melt combined by filtering the corresponding rows with DataFrame.loc. For instance:

In [218]:
df45 = pd.DataFrame({'col': ["A", "A", "B", "B"],
                     'A': [80, 23, np.nan, 22],
                     'B': [80, 55, 76, 67]})
df45

,col,A,B
0,A,80.0,80
1,A,23.0,55
2,B,NaN,76
3,B,22.0,67


In [224]:
melt45 = df45.melt('col')
melt45 = melt45.loc[melt45['col'] == melt45['variable'], 'value']
melt45.reset_index(drop=True)

0    80.0
1    23.0
2    76.0
3    67.0
Name: value, dtype: float64

Formerly this could be achieved with the dedicated DataFrame.lookup method which was deprecated in version 1.2.0.

## Index objects
The pandas Index class and its subclasses can be viewed as implementing an ordered multiset. Duplicates are allowed. However, if you try to convert an Index object with duplicate entries into a set, an exception will be raised.

Index also provides the infrastructure necessary for lookups, data alignment, and reindexing. The easiest way to create an Index directly is to pass a list or other sequence to Index:

In [220]:
index46 = pd.Index(['e', 'd', 'a', 'b'])
index46

Index(['e', 'd', 'a', 'b'], dtype='object')

In [222]:
'd' in index46

True

You can also pass a name to be stored in the index:

In [223]:
index47 = pd.Index(['e', 'd', 'a', 'b'], name='something')
index47.name

'something'

The name, if set, will be shown in the console display:

In [225]:
index48 = pd.Index(list(range(5)), name='rows')
columns48 = pd.Index(['A', 'B', 'C'], name='cols')
df48 = pd.DataFrame(np.random.randn(5, 3), index=index48, columns=columns48)
df48

cols,A,B,C
rows,,,
0,0.712662,-0.101908,0.060274
1,1.571263,2.400797,-1.424809
2,-0.661268,-0.929210,-0.423420
3,-1.081804,-1.025172,0.241770
4,1.228755,-1.032773,-2.080466


In [226]:
df48['A']

rows
0    0.712662
1    1.571263
2   -0.661268
3   -1.081804
4    1.228755
Name: A, dtype: float64

## Setting metadata
Indexes are “mostly immutable”, but it is possible to set and change their name attribute. You can use the rename, set_names to set these attributes directly, and they default to returning a copy.

See Advanced Indexing for usage of MultiIndexes.

In [227]:
ind = pd.Index([1, 2, 3])
ind.rename("apple")

Int64Index([1, 2, 3], dtype='int64', name='apple')

In [228]:
ind

Int64Index([1, 2, 3], dtype='int64')

In [229]:
ind.set_names(["apple"], inplace=True)
ind.name = "bob"
ind

Int64Index([1, 2, 3], dtype='int64', name='bob')

set_names, set_levels, and set_codes also take an optional level argument

In [230]:
index = pd.MultiIndex.from_product([range(3), ['one', 'two']], names=['first', 'second'])
index

MultiIndex([(0, 'one'),
            (0, 'two'),
            (1, 'one'),
            (1, 'two'),
            (2, 'one'),
            (2, 'two')],
           names=['first', 'second'])

In [231]:
index.levels[1]

Index(['one', 'two'], dtype='object', name='second')

In [232]:
index.set_levels(["a", "b"], level=1)

MultiIndex([(0, 'a'),
            (0, 'b'),
            (1, 'a'),
            (1, 'b'),
            (2, 'a'),
            (2, 'b')],
           names=['first', 'second'])

### Set operations on Index objects
The two main operations are union and intersection. Difference is provided via the .difference() method.

In [233]:
a = pd.Index(['c', 'b', 'a'])
b = pd.Index(['c', 'e', 'd'])
a.difference(b)

Index(['a', 'b'], dtype='object')

Also available is the symmetric_difference operation, which returns elements that appear in either idx1 or idx2, but not in both. This is equivalent to the Index created by idx1.difference(idx2).union(idx2.difference(idx1)), with duplicates dropped.

In [234]:
idx1 = pd.Index([1, 2, 3, 4])
idx2 = pd.Index([2, 3, 4, 5])
idx1.symmetric_difference(idx2)

Int64Index([1, 5], dtype='int64')

>Note
>
>The resulting index from a set operation will be sorted in ascending order.

When performing Index.union() between indexes with different dtypes, the indexes must be cast to a common dtype. Typically, though not always, this is object dtype. The exception is when performing a union between integer and float data. In this case, the integer values are converted to float

In [235]:
idx1 = pd.Index([0, 1, 2])
idx2 = pd.Index([0.5, 1.5])
idx1.union(idx2)

Float64Index([0.0, 0.5, 1.0, 1.5, 2.0], dtype='float64')

### Missing values
>Important
>
>Even though Index can hold missing values (NaN), it should be avoided if you do not want any unexpected results. For example, some operations exclude missing values implicitly.

Index.fillna fills missing values with specified scalar value.

In [236]:
idx1 = pd.Index([1, np.nan, 3, 4])
idx1

Float64Index([1.0, nan, 3.0, 4.0], dtype='float64')

In [237]:
idx1.fillna(2)

Float64Index([1.0, 2.0, 3.0, 4.0], dtype='float64')

In [238]:
idx2 = pd.DatetimeIndex([pd.Timestamp('2011-01-01'),
                         pd.NaT,
                         pd.Timestamp('2011-01-03')])
idx2

DatetimeIndex(['2011-01-01', 'NaT', '2011-01-03'], dtype='datetime64[ns]', freq=None)

In [239]:
idx2.fillna(pd.Timestamp('2011-01-02'))

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03'], dtype='datetime64[ns]', freq=None)

## Set / reset index
Occasionally you will load or create a data set into a DataFrame and want to add an index after you’ve already done so. There are a couple of different ways.

### Set an index
DataFrame has a set_index() method which takes a column name (for a regular Index) or a list of column names (for a MultiIndex). To create a new, re-indexed DataFrame:

In [244]:
data = pd.DataFrame({'a': ["bar", "bar", "foo", "foo"],
                     'b': ["one", "two", "one", "two"],
                     'c': ["z", "y", "x", "w"],
                     'd': [1.0, 2.0, 3.0, 4.0],
                    })
data

,a,b,c,d
0,bar,one,z,1.0
1,bar,two,y,2.0
2,foo,one,x,3.0
3,foo,two,w,4.0


In [245]:
indexed1 = data.set_index('c')
indexed1

,a,b,d
c,,,
z,bar,one,1.0
y,bar,two,2.0
x,foo,one,3.0
w,foo,two,4.0


In [246]:
indexed2 = data.set_index(['a', 'b'])
indexed2

c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0

The append keyword option allow you to keep the existing index and append the given columns to a MultiIndex:

In [247]:
frame = data.set_index('c', drop=False)
frame = frame.set_index(['a', 'b'], append=True)
frame

,,,c,d
c,a,b,,
z,bar,one,z,1.0
y,bar,two,y,2.0
x,foo,one,x,3.0
w,foo,two,w,4.0


Other options in set_index allow you not drop the index columns or to add the index in-place (without creating a new object):

In [248]:
data.set_index('c', drop=False)

,a,b,c,d
c,,,,
z,bar,one,z,1.0
y,bar,two,y,2.0
x,foo,one,x,3.0
w,foo,two,w,4.0


In [249]:
data.set_index(['a', 'b'], inplace=True)
data

c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0

### Reset the index
As a convenience, there is a new function on DataFrame called reset_index() which transfers the index values into the DataFrame’s columns and sets a simple integer index. This is the inverse operation of set_index().

In [250]:
data

c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0

In [251]:
 data.reset_index()

,a,b,c,d
0,bar,one,z,1.0
1,bar,two,y,2.0
2,foo,one,x,3.0
3,foo,two,w,4.0


The output is more similar to a SQL table or a record array. The names for the columns derived from the index are the ones stored in the names attribute.

You can use the level keyword to remove only a portion of the index:

In [252]:
frame

,,,c,d
c,a,b,,
z,bar,one,z,1.0
y,bar,two,y,2.0
x,foo,one,x,3.0
w,foo,two,w,4.0


In [253]:
frame.reset_index(level=1)

,,a,c,d
c,b,,,
z,one,bar,z,1.0
y,two,bar,y,2.0
x,one,foo,x,3.0
w,two,foo,w,4.0


reset_index takes an optional parameter drop which if true simply discards the index, instead of putting index values in the DataFrame’s columns.

### Adding an ad hoc index
If you create an index yourself, you can just assign it to the index field:
```python
data.index = index
```

## Returning a view versus a copy
When setting values in a pandas object, care must be taken to avoid what is called chained indexing. Here is an example.

In [255]:
dfmi = pd.DataFrame([list('abcd'),
                     list('efgh'),
                     list('ijkl'),
                     list('mnop')],
                    columns=pd.MultiIndex.from_product([['one', 'two'],
                                                        ['first', 'second']]))
dfmi

one          two       
  first second first second
0     a      b     c      d
1     e      f     g      h
2     i      j     k      l
3     m      n     o      p

Compare these two access methods:

In [256]:
dfmi['one']['second']

0    b
1    f
2    j
3    n
Name: second, dtype: object

In [257]:
dfmi.loc[:, ('one', 'second')]

0    b
1    f
2    j
3    n
Name: (one, second), dtype: object

These both yield the same results, so which should you use? It is instructive to understand the order of operations on these and why method 2 (.loc) is much preferred over method 1 (chained []).

dfmi['one'] selects the first level of the columns and returns a DataFrame that is singly-indexed. Then another Python operation dfmi_with_one['second'] selects the series indexed by 'second'. This is indicated by the variable dfmi_with_one because pandas sees these operations as separate events. e.g. separate calls to __getitem__, so it has to treat them as linear operations, they happen one after another.

Contrast this to df.loc[:,('one','second')] which passes a nested tuple of (slice(None),('one','second')) to a single call to __getitem__. This allows pandas to deal with this as a single entity. Furthermore this order of operations can be significantly faster, and allows one to index both axes if so desired.

### Why does assignment fail when using chained indexing?
The problem in the previous section is just a performance issue. What’s up with the SettingWithCopy warning? We don’t usually throw warnings around when you do something that might cost a few extra milliseconds!

But it turns out that assigning to the product of chained indexing has inherently unpredictable results. To see this, think about how the Python interpreter executes this code:

```python
dfmi.loc[:, ('one', 'second')] = value
# becomes
dfmi.loc.__setitem__((slice(None), ('one', 'second')), value)
```

But this code is handled differently:

```python
dfmi['one']['second'] = value
# becomes
dfmi.__getitem__('one').__setitem__('second', value)
```

See that __getitem__ in there? Outside of simple cases, it’s very hard to predict whether it will return a view or a copy (it depends on the memory layout of the array, about which pandas makes no guarantees), and therefore whether the __setitem__ will modify dfmi or a temporary object that gets thrown out immediately afterward. That’s what SettingWithCopy is warning you about!

>Note
>
>You may be wondering whether we should be concerned about the loc property in the first example. But dfmi.loc is guaranteed to be dfmi itself with modified indexing behavior, so dfmi.loc.__getitem__ / dfmi.loc.__setitem__ operate on dfmi directly. Of course, dfmi.loc.__getitem__(idx) may be a view or a copy of dfmi.

Sometimes a SettingWithCopy warning will arise at times when there’s no obvious chained indexing going on. These are the bugs that SettingWithCopy is designed to catch! pandas is probably trying to warn you that you’ve done this:

In [259]:
def do_something(df):
    foo = df[['bar', 'baz']]  # Is foo a view? A copy? Nobody knows!
    # ... many lines here ...
    # We don't know whether this will modify df or not!
    foo['quux'] = value
    return foo

Yikes!

### Evaluation order matters
When you use chained indexing, the order and type of the indexing operation partially determine whether the result is a slice into the original object, or a copy of the slice.

pandas has the SettingWithCopyWarning because assigning to a copy of a slice is frequently not intentional, but a mistake caused by chained indexing returning a copy where a slice was expected.

If you would like pandas to be more or less trusting about assignment to a chained indexing expression, you can set the option mode.chained_assignment to one of these values:

* 'warn', the default, means a SettingWithCopyWarning is printed.

* 'raise' means pandas will raise a SettingWithCopyException you have to deal with.

* None will suppress the warnings entirely.

In [260]:
dfb = pd.DataFrame({'a': ['one', 'one', 'two',
                          'three', 'two', 'one', 'six'],
                    'c': np.arange(7)})

# This will show the SettingWithCopyWarning
# but the frame values will be set
dfb['c'][dfb['a'].str.startswith('o')] = 42

<ipython-input-260-c538906c508c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb['c'][dfb['a'].str.startswith('o')] = 42


This however is operating on a copy and will not work.

```shell
>>> pd.set_option('mode.chained_assignment','warn')
>>> dfb[dfb['a'].str.startswith('o')]['c'] = 42
Traceback (most recent call last)
     ...
SettingWithCopyWarning:
     A value is trying to be set on a copy of a slice from a DataFrame.
     Try using .loc[row_index,col_indexer] = value instead
```

A chained assignment can also crop up in setting in a mixed dtype frame.

>Note
>
>These setting rules apply to all of .loc/.iloc.

The following is the recommended access method using .loc for multiple items (using mask) and a single item using a fixed index:

In [261]:
dfc = pd.DataFrame({'a': ['one', 'one', 'two',
                          'three', 'two', 'one', 'six'],
                    'c': np.arange(7)})
dfd = dfc.copy()

# Setting multiple items using a mask
mask = dfd['a'].str.startswith('o')
dfd.loc[mask, 'c'] = 42
dfd

,a,c
0,one,42
1,one,42
2,two,2
3,three,3
4,two,4
5,one,42
6,six,6


In [262]:
# Setting a single item
dfd = dfc.copy()
dfd.loc[2, 'a'] = 11
dfd

,a,c
0,one,0
1,one,1
2,11,2
3,three,3
4,two,4
5,one,5
6,six,6


The following can work at times, but it is not guaranteed to, and therefore should be avoided:

In [263]:
dfd = dfc.copy()
dfd['a'][2] = 111
dfd

<ipython-input-263-43c82c6063b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfd['a'][2] = 111


,a,c
0,one,0
1,one,1
2,111,2
3,three,3
4,two,4
5,one,5
6,six,6


Last, the subsequent example will not work at all, and so should be avoided:

```python
>>> pd.set_option('mode.chained_assignment','raise')
>>> dfd.loc[0]['a'] = 1111
Traceback (most recent call last)
     ...
SettingWithCopyException:
     A value is trying to be set on a copy of a slice from a DataFrame.
     Try using .loc[row_index,col_indexer] = value instead
```
>Warning
>
>The chained assignment warnings / exceptions are aiming to inform the user of a possibly invalid assignment. There may be false positives; situations where a chained assignment is inadvertently reported.